# Testing the trained flow

Once the Normalising Flow was trained using `train.py`, it can be used for inversion. This is a tutorial on how to generate results from some synthetic test data.

First, we need to read in the flow model.

In [2]:
import torch
from giflow.flowmodel import FlowModel

flow_location = 'trained_flow'

device = torch.device('cuda') # defining the GPU

flow = FlowModel()
flow.load(flow_location)
flow.flowmodel.to(device)
flow.save_location = flow_location
data = flow.data_location

trained_flow


FileNotFoundError: [Errno 2] No such file or directory: 'trained_flow/flow.pt'

Need to load in data that we want to test the network with. In this case we are using the validation data to conduct a P-P test. We need to ensure that the data set that we are testing on has the same parameters as the training data, eg. the same `survey_coordinates_to_include` parameter.

In [ ]:
# Validation data
valsize = 500 
with open(os.path.join(data, 'valset.pkl'), 'rb') as file:
    dt_val = pkl.load(file)
    validation_data, validation_conditional = dt_val.make_data_for_network(
        survey_coordinates_to_include = ['noise_scale'],
        model_info_to_include = [],
        add_noise = True,
        mix_survey_order = False,
    )
validation_dataset = flow.make_tensor_dataset(validation_data, validation_conditional, device=device, scale=True)

# Test data
testsize = 2
with open(os.path.join(data, 'testset.pkl'), 'rb') as file:
    dt_test = pkl.load(file)
    test_data, test_conditional = dt_test.make_data_for_network(
        survey_coordinates_to_include = ['noise_scale'],
        model_info_to_include = [],
        add_noise = True,
        mix_survey_order = False,
    )
test_dataset = flow.make_tensor_dataset(test_data, test_conditional, device=device, scale=True)

In [ ]:
Conducting a P-P test, where diagonal lines indicate statistical consistency.

In [ ]:
# P-P plot
flow.pp_test(validation_dataset=validation_dataset,
             parameter_labels=dt_val.parameter_labels)

Now, we want to generate some inversion results from some test data. For this, we need to sample the flow and construct `FlowResults` objects.

In [ ]:
# Generating results for some test data
results = []
for i in range(testsize):
    samples, log_probabilities = flow.sample_and_logprob(test_dataset.tensors[1][i], # the conditional
                                                         num=2000) # number of samples we want to draw
    result = BoxFlowResults(samples=samples,
                            conditional=[test_conditional[j][i] for j in range(len(test_conditional))],
                            log_probabilities=log_probabilities,
                            true_parameters=np.array([test_data[0][i]]),
                            parameter_labels=dt_val.parameter_labels,
                            survey_coordinates=dt_test.surveys[0].survey_coordinates
                           )
    results.append(result)
    result.directory = os.path.join(flow_location, f"testcase_{i}/")
    
    # plotting the surveys we are inverting
    dt_test.surveys[i].plot_contours(filename=os.path.join(result.directory, "survey.png"),
                                     include_noise=True)

To make corner plots of the posterior probability distribution, we can use the `corner_plot()` function.

In [ ]:
for i, result in enumerate(results):
    result.corner_plot(filename="corner_plot.png")

To make voxelised plots from parameterised data, first we need to define what grid we want to translate our data to. Then, we can use the `plot_compare_voxel_slices()` function.

In [ ]:
dt_test.model_framework['ranges'] = [[-0.75, 0.75], [-0.75, 0.75], [-1.5, 0.0]]
dt_test.model_framework['grid_shape'] = [10, 10, 10]

for i, result in enumerate(results):
    result.plot_compare_voxel_slices(filename=f"compare_voxel_slices.png",
                                     plot_truth=True,
                                     normalisation=[-2500.0, 500.0],
                                     model_framework=dt_test.model_framework,
                                     slice_coords=[1, 4, 8])

There are many other testing and plotting functionalities built into the `FlowResults` class.